In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
import os
load_dotenv()

# reference: https://github.com/langchain-ai/langchain-mcp/blob/main/examples/langgraph-mcp.ipynb

client = MultiServerMCPClient(
    {
        "github": {
            "command": "docker",
            # Replace with absolute path to your math_server.py file
            "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server"],
            "env": {
                "GITHUB_PERSONAL_ACCESS_TOKEN": os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN"),
                **os.environ,
            },
            "transport": "stdio",
        }
    }
)


In [ ]:
tools = await client.get_tools()
agent = create_react_agent(
    "openai:gpt-4.1",
    tools
)

github_response = await agent.ainvoke(
    {"messages": [{"role": "user", "content": "List out public repo in my github account"}]}
)


In [ ]:
print(github_response["messages"][-1].content)

---

In [ ]:
# Create server parameters for stdio connection
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

server_params = StdioServerParameters(
    command="docker",
    # Make sure to update to the full absolute path to your math_server.py file
    args=[
    "run",
    "-i",
    "--rm",
    "-e",
    "GITHUB_PERSONAL_ACCESS_TOKEN",
    "ghcr.io/github/github-mcp-server"],
    env={
        "GITHUB_PERSONAL_ACCESS_TOKEN": os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN"),
        **os.environ,
    },
    transport="stdio",
)

async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        # Initialize the connection
        await session.initialize()

        # Get tools
        tools = await load_mcp_tools(session)

        # Create and run the agent
        agent = create_react_agent("openai:gpt-4.1", tools)
        agent_response = await agent.ainvoke({"messages": "List out public repo in my github account"})

In [ ]:
print(agent_response["messages"][-1].content)